In [2]:
import pandas as pd
import numpy as np
df_customer = pd.read_csv('customer_profile.csv')
df_consumption = pd.read_csv('consumption_log.csv')
df_tariff = pd.read_csv('price_tariff.csv')
print(f"Đã tải thành công 3 bảng dữ liệu: Khách hàng ({len(df_customer)} dòng), Tiêu thụ ({len(df_consumption)} dòng), Bảng giá ({len(df_tariff)} dòng).")


# customer_id: Chuyển HOA và xóa khoảng trắng
df_customer['customer_id'] = df_customer['customer_id'].astype(str).str.upper().str.strip()

# house_type: Chuẩn hóa tên loại nhà
df_customer['house_type'] = (df_customer['house_type']
                             .astype(str).str.strip().str.lower()
                             .replace({'chung cu': 'Chung Cư', 'tap the': 'Tập Thể',
                                       'biet thu': 'Biệt Thự', 'nha pho': 'Nhà Phố'})
                             .str.title()
                            )

# region: Chuẩn hóa tên khu vực/tỉnh thành
df_customer['region'] = (df_customer['region']
                         .astype(str).str.strip().str.lower()
                         .replace({'ha noi': 'Hà Nội', 'tp.hcm': 'TP. Hồ Chí Minh', 
                                   'tp hcm': 'TP. Hồ Chí Minh', 'bac ninh': 'Bắc Ninh',
                                   'bac  ninh': 'Bắc Ninh', 'da nang': 'Đà Nẵng'})
                         .str.title()
                        )
df_customer_clean = df_customer
print("\n[customer_profile - 5 dòng đã làm sạch]:")
print(df_customer_clean.head())

# customer_id & meter_id: Chuyển HOA và xóa khoảng trắng
df_consumption['customer_id'] = df_consumption['customer_id'].astype(str).str.upper().str.strip()
df_consumption['meter_id'] = df_consumption['meter_id'].astype(str).str.upper().str.strip()

# reading_date: Chuyển về định dạng datetime thống nhất (xử lý cả DD/MM/YYYY và YYYY-MM-DD)
df_consumption['reading_date'] = pd.to_datetime(df_consumption['reading_date'], errors='coerce', dayfirst=True)

# consumption_kwh: Loại bỏ text, xử lý giá trị âm/NaN
df_consumption['consumption_kwh'] = (df_consumption['consumption_kwh']
                                     .astype(str)
                                     .str.replace(r'[^\d\.-]', '', regex=True) # Chỉ giữ số, dấu chấm, dấu trừ
                                     .str.strip()
                                    )
df_consumption['consumption_kwh'] = pd.to_numeric(df_consumption['consumption_kwh'], errors='coerce')
# Xử lý giá trị âm bất thường (thay bằng NaN)
df_consumption.loc[df_consumption['consumption_kwh'] < 0, 'consumption_kwh'] = np.nan
df_consumption_clean = df_consumption.dropna(subset=['reading_date', 'consumption_kwh']).reset_index(drop=True)
print("\n[consumption_log - 5 dòng đã làm sạch]:")
print(df_consumption_clean.head())

# Loại bỏ text và chuyển đổi sang dạng số
df_tariff['kwh_min'] = df_tariff['kwh_min'].astype(str).str.replace(r'[^\d]', '', regex=True).str.strip()
df_tariff['kwh_max'] = df_tariff['kwh_max'].astype(str).str.replace(r'[^\d]', '', regex=True).str.strip()
df_tariff['price_per_kwh'] = df_tariff['price_per_kwh'].astype(str).str.replace(r'[^\d]', '', regex=True).str.strip()
df_tariff['kwh_min'] = pd.to_numeric(df_tariff['kwh_min'], errors='coerce')
df_tariff['kwh_max'] = pd.to_numeric(df_tariff['kwh_max'], errors='coerce')
df_tariff['price_per_kwh'] = pd.to_numeric(df_tariff['price_per_kwh'], errors='coerce')

# Xử lý bậc thang cuối có kwh_max bị trống: đặt là vô hạn (np.inf)
df_tariff.loc[df_tariff['tariff_level'] == 'B5', 'kwh_max'] = np.inf
df_tariff_clean = df_tariff.dropna(subset=['kwh_min', 'price_per_kwh']).reset_index(drop=True)
print("\n[price_tariff - Bảng giá đã làm sạch]:")
print(df_tariff_clean)


#Xuất các file sạch để dùng cho các nhiệm vụ sau
print("\n--- Xuất file sạch (Clean Files) ---")
# Xuất file thông tin khách hàng đã chuẩn hóa
df_customer_clean.to_csv('customer_profile_clean.csv', index=False)
print("-> Đã xuất file: customer_profile_clean.csv")
# Xuất file tiêu thụ đã sạch
df_consumption_clean.to_csv('consumption_log_clean.csv', index=False)
print("-> Đã xuất file: consumption_log_clean.csv")
# Xuất file biểu giá đã chuẩn hóa
df_tariff_clean.to_csv('price_tariff_clean.csv', index=False)
print("-> Đã xuất file: price_tariff_clean.csv")
print("\n Hoàn thành việc làm sạch và xuất file.")



Đã tải thành công 3 bảng dữ liệu: Khách hàng (40 dòng), Tiêu thụ (150 dòng), Bảng giá (5 dòng).

[customer_profile - 5 dòng đã làm sạch]:
  customer_id house_type           region
0       KH100   Chung Cư           Hà Nội
1       KH101   Biệt Thự           Hà Nội
2       KH102   Chung Cư           Hà Nội
3       KH103    Tập Thể  Tp. Hồ Chí Minh
4       KH104    Tập Thể         Bắc Ninh

[consumption_log - 5 dòng đã làm sạch]:
  customer_id meter_id reading_date  consumption_kwh
0       KH136    CT016   2024-01-27             50.0
1       KH126    CT999   2024-03-06            150.0
2       KH103    CT007   2024-02-29             75.0
3       KH113    CT009   2024-03-23            100.0
4       KH138    CT008   2024-01-15             50.0

[price_tariff - Bảng giá đã làm sạch]:
  tariff_level  kwh_min  kwh_max  price_per_kwh
0           B1        0     50.0           1800
1           B2       51    100.0           2000
2           B3      101    200.0           2500
3           B4     

## Nhiệm Vụ 4

## Merge customer_profile với consumption_log theo customer_id

In [3]:
print("\n Hợp nhất Dữ liệu Khách hàng và Tiêu thụ ")
df_merged = pd.merge(df_consumption_clean, df_customer_clean, on='customer_id', how='left')
print(f"Số lượng bản ghi sau khi hợp nhất: {len(df_merged)}")


 Hợp nhất Dữ liệu Khách hàng và Tiêu thụ 
Số lượng bản ghi sau khi hợp nhất: 66


## Xác định bậc thang giá tương ứng cho mỗi bản ghi consumption_kwh dựa trên price_tariff

In [4]:
#Xác định bậc thang giá tương ứng cho mỗi bản ghi consumption_kwh dựa trên price_tariff
print("\n Xác định Bậc thang giá và Tính Chi phí Ước tính")
df_merged['tariff_level'] = None
df_merged['price_per_kwh'] = 0
# Vòng lặp gán bậc thang giá dựa trên mức tiêu thụ
for index, row in df_tariff_clean.iterrows():
    kwh_min = row['kwh_min']
    kwh_max = row['kwh_max']
    level = row['tariff_level']
    price = row['price_per_kwh']
    condition = (df_merged['consumption_kwh'] >= kwh_min) & (df_merged['consumption_kwh'] <= kwh_max)
    # Gán bậc thang và giá cho các bản ghi thỏa mãn điều kiện
    df_merged.loc[condition, 'tariff_level'] = level
    df_merged.loc[condition, 'price_per_kwh'] = price
# Tính toán Chi phí ước tính cho từng kỳ 
df_merged['estimated_cost'] = df_merged['consumption_kwh'] * df_merged['price_per_kwh']
# Hoàn thiện DataFrame và làm tròn cột tiền
df_complete = df_merged
df_complete['estimated_cost'] = df_complete['estimated_cost'].round(0).astype(int) 
print("Đã hoàn thành xác định Bậc thang giá và tính Estimated Cost.")


 Xác định Bậc thang giá và Tính Chi phí Ước tính
Đã hoàn thành xác định Bậc thang giá và tính Estimated Cost.


## Phát hiện các bản ghi không tìm được bậc thang phù hợp hoặc mã khách hàng không tồn tại

In [5]:
#Phát hiện các bản ghi không tìm được bậc thang phù hợp hoặc mã khách hàng không tồn taị
print("--- 1. Kiểm tra Bản ghi Thiếu Hồ sơ Khách hàng ---")
missing_customer_profile = df_complete[df_complete['house_type'].isna()]
if missing_customer_profile.empty:
    print(" Không tìm thấy bản ghi tiêu thụ nào thiếu thông tin hồ sơ khách hàng (Mã KH tồn tại).")
else:
    print(f" Phát hiện {len(missing_customer_profile)} bản ghi tiêu thụ có Mã khách hàng không tồn tại trong file hồ sơ.")
    print("Các bản ghi bị thiếu hồ sơ:")
    print(missing_customer_profile[['customer_id', 'reading_date', 'consumption_kwh']].head())
print("\n--- 2. Kiểm tra Bản ghi Không tìm được Bậc thang phù hợp ---")
unmatched_tariff = df_complete[df_complete['price_per_kwh'] == 0]
if unmatched_tariff.empty:
    print("Tất cả các bản ghi tiêu thụ đều đã được gán bậc thang giá thành công.")
else:
    print(f"Phát hiện {len(unmatched_tariff)} bản ghi tiêu thụ không tìm được bậc thang giá phù hợp.")
    print("Các bản ghi không khớp bậc thang (Mức tiêu thụ ngoài khoảng [0 - 300+]):")
    print(unmatched_tariff[['customer_id', 'reading_date', 'consumption_kwh', 'price_per_kwh', 'tariff_level']].head())

--- 1. Kiểm tra Bản ghi Thiếu Hồ sơ Khách hàng ---
 Không tìm thấy bản ghi tiêu thụ nào thiếu thông tin hồ sơ khách hàng (Mã KH tồn tại).

--- 2. Kiểm tra Bản ghi Không tìm được Bậc thang phù hợp ---
Tất cả các bản ghi tiêu thụ đều đã được gán bậc thang giá thành công.


## Xuất bảng dữ liệu hoàn chỉnh phục vụ phân tích tiếp theo

In [8]:
# Xuất bảng dữ liệu hoàn chỉnh phục vụ phân tích tiếp theo. 
print("\n--- BẢNG DỮ LIỆU HOÀN CHỈNH (df_complete) ---")
print(f"Tổng số bản ghi: {len(df_complete)}")
print("Dữ liệu (Hiển thị 10 bản ghi đầu tiên):")
print(df_complete.head(10))
print(df_complete.to_csv('complete_data.csv', index=False))


--- BẢNG DỮ LIỆU HOÀN CHỈNH (df_complete) ---
Tổng số bản ghi: 66
Dữ liệu (Hiển thị 10 bản ghi đầu tiên):
  customer_id meter_id reading_date  consumption_kwh house_type  \
0       KH136    CT016   2024-01-27             50.0   Biệt Thự   
1       KH126    CT999   2024-03-06            150.0    Nhà Phố   
2       KH103    CT007   2024-02-29             75.0    Tập Thể   
3       KH113    CT009   2024-03-23            100.0    Tập Thể   
4       KH138    CT008   2024-01-15             50.0   Chung Cư   
5       KH102    CT006   2024-01-18            150.0   Chung Cư   
6       KH117    CT999   2024-04-19            100.0   Biệt Thự   
7       KH115    CT014   2024-01-14            120.0   Chung Cư   
8       KH128    CT999   2024-01-08            120.0    Tập Thể   
9       KH107    CT008   2024-02-19            300.0    Tập Thể   

            region tariff_level  price_per_kwh  estimated_cost  
0  Tp. Hồ Chí Minh           B1           1800           90000  
1           Hà Nội       